In [1]:
from keras.datasets import boston_housing
from keras.datasets import reuters
import numpy as np
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import matplotlib.pyplot as plt
import time
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

Using TensorFlow backend.


In [3]:
df = pd.read_excel(r'C:\Users\Big data\Desktop\class\funcardproject\斷詞與和卡額度_20群.xls',encoding='utf-16')
df = df.loc[:, ["age","serveTime","credLimit","Loan","SalPerY","holdCard","Career","credLimit_group"]] 
len(df)

1687

In [4]:
#如果某一行有空值,則整行去除
df =df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )

In [5]:
#因為我們要預測的是分群,因此要把浮點改成INT
df['credLimit_group'] = df['credLimit_group'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 1686
Data columns (total 8 columns):
age                1680 non-null float64
serveTime          1680 non-null float64
credLimit          1680 non-null int64
Loan               1680 non-null float64
SalPerY            1680 non-null int64
holdCard           1680 non-null int64
Career             1680 non-null int64
credLimit_group    1680 non-null int32
dtypes: float64(3), int32(1), int64(4)
memory usage: 111.6 KB


In [6]:
# train_test_split是一個自動切分訓練資料與測試資料的套件,test_size是訓練與樣本的切分比例,random_state是r的SEED
train_data, test_data, train_targets, test_targets = train_test_split(df.loc[:, ["age","serveTime","Loan","SalPerY","holdCard","Career"]] , df.loc[:, ["credLimit_group"]] , test_size=0.3, random_state=42)


In [7]:
# 標準化= (x-x.mean)/std.x
# 也可用正規化,但我測起來機器學習用標準化比較準
mean = train_data.mean(axis=0)
train_data -=mean
std = train_data.std(axis=0)
train_data/=std
test_data-=mean
test_data/=std
#正常情況為防止train data資訊洩漏給test data 兩邊的mean,std要分開做,但我懶所以..

In [12]:
# n_estimators有幾棵樹
# max_depth枝葉有多茂密
# random_state取樣
rf = RandomForestClassifier(n_estimators=5, max_depth=8,
                              random_state=1)

# 建模 步驟一樣

In [15]:
rf.fit(train_data,train_targets)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [16]:
訓練準確度 = rf.score(train_data, train_targets)
驗證準確度 = rf.score(test_data, test_targets)
print("訓練準確度:",訓練準確度,"%")
print("驗證準確度:",驗證準確度,"%")

訓練準確度: 0.6045918367346939 %
驗證準確度: 0.35119047619047616 %


# 測試 小綠同學

In [17]:
小綠 = pd.DataFrame(columns=["age","serveTime","Loan","SalPerY","holdCard","Career"])
小綠.loc[0]=8,120,4,600000,1,1
小綠.loc[1]=280,120,3,600000000,1,1
小綠.loc[2]=28,12,0,87,2,0
小綠.loc[3]=28,12,0,8700,2,0
小綠

,age,serveTime,Loan,SalPerY,holdCard,Career
0,8,120,4,600000,1,1
1,280,120,3,600000000,1,1
2,28,12,0,87,2,0
3,28,12,0,8700,2,0


In [18]:
# 因為先前輸入時有先標準化,因此輸入也要標準化
小綠-=mean
小綠/=std
#轉array
小綠 = 小綠.astype(float)
# 小綠 = np.array(小綠).astype(float)

In [19]:
# 他給出的是每一群的機率(相加為一)
preds = rf.predict(小綠)#直接回傳分在哪一群
print("四個小綠依序被分在",preds,"群")
print("____________________________________________________________________________________________________")
pred = rf.predict_proba(小綠)#如果你想看他的機率分布
print(pred)

四個小綠依序被分在 [3 7 0 0] 群
____________________________________________________________________________________________________
[[0.2        0.2        0.22352941 0.28235294 0.02352941 0.02352941
  0.04705882 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.03980344 0.04520885 0.08157248
  0.05257985 0.44864865 0.         0.01621622 0.03439803 0.
  0.22702703 0.         0.         0.05454545 0.        ]
 [0.36       0.04       0.         0.23333333 0.         0.03333333
  0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.36       0.04       0.         0.23333333 0.         0.03333333
  0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]]


In [20]:
小綠群=[print("第",str(i),"個小綠被分在第",preds[i],"群") for i in range(len(小綠))]

第 0 個小綠被分在第 3 群
第 1 個小綠被分在第 7 群
第 2 個小綠被分在第 0 群
第 3 個小綠被分在第 0 群


In [112]:
# 逤要做複雜 可參考https://github.com/abhinavsagar/ICC-2019-WC-prediction/blob/master/ICC%202019%20WC%20RF.ipynb